# Collaborative Filtering: k-Nearest Neighbor

먼저 필요한 라이브러리를 설치해야 한다.<br><br>
사전 설치가 필요한 Library 리스트<br>
- pip install pandas: 데이터프레임 처리 라이브러리<br>
- pip install numpy: array 처리 라이브러리<br>
- pip install codecs: 파일 불러오기 라이브러리<br>
- pip install suprise: 추천시스템 모델링 라이브러리<br>

분석에 필요한 라이브러리를 불러온다.

In [1]:
import pandas as pd
import codecs
import surprise

ModuleNotFoundError: No module named 'surprise'

In [ ]:
!pip install surprise

<br>

# KNN을 활용한 영화 추천 

## 데이터 처리

이 실습에서는 KNN을 활용해 영화를 추천한다.<br><br>
MovieLens 영화 추천 웹사이트에서 제공하는 평점 데이터를 기반으로 영화 추천을 진행한다.<br>
추천에 필요한 평점, 영화 정보를 불러와 데이터프레임을 구성하며, 각 데이터는 아래의 정보를 포함하고 있다.<br>
- ratings.dat: UserID, MovieID, Rating, Timestamp
- movies.dat: MovieID, Title, Genres

In [2]:
ratings_list = [i.strip().split("::") for i in codecs.open('./data/ml-1m/ratings.dat', 'r', encoding='latin').readlines()]
movies_list = [i.strip().split("::") for i in codecs.open('./data/ml-1m/movies.dat', 'r', encoding='latin').readlines()]

리스트 형태의 데이터를 데이터프레임 형태로 변환하고, 평점은 string에서 numeric으로 type을 변경해준다.

In [3]:
ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
ratings_df['Rating'] = ratings_df['Rating'].apply(pd.to_numeric)

평점 데이터프레임은 다음과 같다.<br>
예를 들어 1번 user는 1193번 영화에 평점 5점을 주었다.

In [4]:
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


평점 데이터프레임 중 영화 추천에 불필요한 정보인 Timestamp 열을 삭제해 최종 데이터를 구축한다.

In [5]:
del ratings_df['Timestamp']
ratings_df.head()

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


영화 데이터프레임은 다음과 같다.

In [6]:
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


추천 시스템 라이브러리 surprise를 사용하기 위해 구축한 ratings_df 데이터프레임을 해당 라이브러리에서 요구하는 input 형태에 맞게 변형한다.<br>
surprise 라이브러리는 dataset을 인풋으로 받으며, pandas 데이터프레임으로부터 dataset을 로딩하기 위해 Reader object와 load_from_df() method를 사용한다.

In [7]:
from surprise import Reader
from surprise import Dataset

In [8]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_df, reader)

<br>

## KNN 모델링

KNN 모델링에 필요한 라이브러리를 불러온다.

In [9]:
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise import accuracy

<br>

KNN에서 user 벡터 (평점 행렬의 행 벡터)나 item 벡터 (평점 행렬의 열 벡터)의 유사도를 비교하기 위해 다양한 지표를 사용할 수 있다.<br>
suprise.KNNBasic 함수에서는 아래와 같은 유사도 기준을 제공하며, 본 강의에서는 유클리드 거리와 코사인 유사도를 중심으로 분석을 진행한다.<br>
- 유클리드 거리 기반 유사도 (Mean Squared Difference Similarity)
- 코사인 유사도 (Cosine Similarity)
- 피어슨 유사도 (Pearson Similarity)
- 피어슨-베이스라인 유사도 (Pearson-Baseline Similarity)

surprise.KNNBasic 패키지의 유사도 설정 옵션은 다음과 같다.<br>
- name: 사용할 유사도의 종류. ['msd', 'cosine', 'pearson', 'pearson_baseline'] 중 선택.
- user_based: True면 사용자 기반, False면 상품 기반.
- min_support: 두 사용자나, 상품에서 공통적으로 있는 평점 원소의 수의 최솟값. 공통 평점 원소의 수가 이 값보다 적으면 해당 벡터는 사용하지 않는다. 디폴트는 1.
- shrinkage: Shrinkage 가중치. 디폴트는 100.

KNN의 user/item 기반, 유클리드거리/코사인유사도 옵션의 조합을 이용해 아래와 같이 4가지 옵션 리스트를 생성한다.

In [10]:
sim_options = [{'name': 'msd', 'user_based': True},
               {'name': 'cosine', 'user_based': True},
               {'name': 'msd', 'user_based': False},
               {'name': 'cosine', 'user_based': False}]

위에서 생성한 4가지 옵션 리스트를 기반으로 4가지 모델을 만들고 cross-validation 결과를 통해 가장 성능이 좋은 모델을 선택한다.

In [11]:
def model_selection(sim_options):
    perf = []
    # iterate over all algorithms
    for i, sim_option in enumerate(sim_options):
        print('==> ', i, '번째 모델 학습을 시작합니다.')
        
        # make KNN model for each option
        algo = KNNBasic(sim_options=sim_option)

        # perform cross validation
        results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

        # get results
        tmp = pd.DataFrame.from_dict(results).mean(axis=0)
        tmp = tmp.append(pd.Series([sim_option['name'], sim_option['user_based']], index=['metric', 'user_based']))
        perf.append(tmp)

    model_selection_results = pd.DataFrame(perf).set_index(['metric', 'user_based']).sort_values('test_rmse')
    return model_selection_results

In [12]:
model_selection_results = model_selection(sim_options)

==>  0 번째 모델 학습을 시작합니다.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
==>  1 번째 모델 학습을 시작합니다.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
==>  2 번째 모델 학습을 시작합니다.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
==>  3 번째 모델 학습을 시작합니다.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [13]:
model_selection_results

test_rmse   fit_time   test_time
metric user_based                                  
msd    False        0.925074   7.089707   72.232902
       True         0.930937  22.562656  146.383257
cosine True         0.979274  46.451010  146.975886
       False        1.003462  14.945996   72.855004

<br>

## 최적 KNN 모델 분석

위에서 실험한 4가지 모델 중 test 데이터에 대한 rmse 값을 기준으로 유클리드 거리를 사용한 item 기반 KNN 알고리즘이 가장 좋은 결과를 도출했다.<br>
따라서 해당 알고리즘을 이용해 보다 자세한 결과를 확인하기 위해 데이터를 7:3의 비율로 train과 test 데이터로 분할한다.

In [14]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

앞서 model selection에서 선택된 유클리드 거리를 사용한 item 기반 KNN 알고리즘을 구축하고 해당 모델을 train 데이터로 학습한다.

In [15]:
sim_option = {'name': 'msd', 'user_based':False}
model_knn = KNNBasic(sim_options=sim_option)
model_knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


train 데이터로 학습한 KNN 알고리즘을 기반으로 test 데이터의 평점을 예측한다.

In [16]:
preds = model_knn.test(testset)
preds

[Prediction(uid='3718', iid='1234', r_ui=3.0, est=3.762350718741624, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='4048', iid='1937', r_ui=5.0, est=4.077290640722361, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='5074', iid='1458', r_ui=1.0, est=3.0832951945080094, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='5502', iid='608', r_ui=4.0, est=3.965104242805639, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='1903', iid='596', r_ui=4.0, est=3.9736913763297936, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='1116', iid='1220', r_ui=4.0, est=4.146668824382584, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='646', iid='2406', r_ui=5.0, est=3.946848744435971, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='4972', iid='1994', r_ui=3.0, est=4.046743159222435, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='699', iid='1453'

In [17]:
accuracy.rmse(preds)

RMSE: 0.9220


0.9219529099076172

test 데이터 중 예측값과 실제값의 차이가 작은 상위 10개의 best prediction을 보다 상세하게 살펴보면 다음과 같다.

In [18]:
preds_df = pd.DataFrame(preds)
preds_df.head()

,uid,iid,r_ui,est,details
0,3718,1234,3.0,3.762351,"{'actual_k': 40, 'was_impossible': False}"
1,4048,1937,5.0,4.077291,"{'actual_k': 40, 'was_impossible': False}"
2,5074,1458,1.0,3.083295,"{'actual_k': 40, 'was_impossible': False}"
3,5502,608,4.0,3.965104,"{'actual_k': 40, 'was_impossible': False}"
4,1903,596,4.0,3.973691,"{'actual_k': 40, 'was_impossible': False}"


preds_df 데이터프레임에 예측값과 실제값의 차이를 나타내는 err 열을 추가한다.

In [19]:
preds_df['err'] = abs(preds_df.est - preds_df.r_ui)
preds_df.head()

,uid,iid,r_ui,est,details,err
0,3718,1234,3.0,3.762351,"{'actual_k': 40, 'was_impossible': False}",0.762351
1,4048,1937,5.0,4.077291,"{'actual_k': 40, 'was_impossible': False}",0.922709
2,5074,1458,1.0,3.083295,"{'actual_k': 40, 'was_impossible': False}",2.083295
3,5502,608,4.0,3.965104,"{'actual_k': 40, 'was_impossible': False}",0.034896
4,1903,596,4.0,3.973691,"{'actual_k': 40, 'was_impossible': False}",0.026309


err를 기준으로 실제값의 차이가 작은 상위 10개의 best prediction을 살펴본다.

In [20]:
best_preds = preds_df.sort_values(by='err')[:10]
best_preds

,uid,iid,r_ui,est,details,err
99126,4169,2350,3.0,3.0,"{'actual_k': 40, 'was_impossible': False}",0.0
3876,3598,858,1.0,1.0,"{'actual_k': 40, 'was_impossible': False}",0.0
107392,3598,1617,1.0,1.0,"{'actual_k': 40, 'was_impossible': False}",0.0
179132,3324,1148,5.0,5.0,"{'actual_k': 13, 'was_impossible': False}",0.0
34338,3610,1237,1.0,1.0,"{'actual_k': 40, 'was_impossible': False}",0.0
140417,4016,200,3.0,3.0,"{'actual_k': 40, 'was_impossible': False}",0.0
289653,3902,3793,5.0,5.0,"{'actual_k': 40, 'was_impossible': False}",0.0
297775,3902,1304,5.0,5.0,"{'actual_k': 40, 'was_impossible': False}",0.0
224289,2744,714,1.0,1.0,"{'actual_k': 40, 'was_impossible': False}",0.0
145176,4277,882,4.0,4.0,"{'actual_k': 40, 'was_impossible': False}",0.0


best prediction 중 10번째 user가 실제로 봤던 영화와 추천받은 영화를 비교하면 아래와 같다.<br>
먼저 10번째 user가 실제로 봤던 영화의 제목과 장르를 살펴본다.

In [21]:
user_MovieID = ratings_df[ratings_df['UserID'] == list(best_preds.uid)[9]]
user_MovieID = user_MovieID['MovieID']
user_MovieID = list(user_MovieID)

In [22]:
user_Movie = movies_df[movies_df['MovieID'].isin(user_MovieID)]
user_Movie

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
5,6,Heat (1995),Action|Crime|Thriller
7,8,Tom and Huck (1995),Adventure|Children's
9,10,GoldenEye (1995),Action|Adventure|Thriller
10,11,"American President, The (1995)",Comedy|Drama|Romance
12,13,Balto (1995),Animation|Children's
13,14,Nixon (1995),Drama
15,16,Casino (1995),Drama|Thriller
17,18,Four Rooms (1995),Thriller


10번째 user가 실제로 즐겨봤던 상위 5개의 영화의 장르를 살펴본다.

In [23]:
user_Genre = {}
for genre in set(user_Movie['Genres']):
    user_Genre[genre] = list(user_Movie['Genres']).count(genre)

In [24]:
user_Genre = sorted(user_Genre.items(), key=lambda x:x[1], reverse=True)
user_Genre[:5]

[('Drama', 315),
 ('Comedy', 133),
 ('Thriller', 68),
 ('Comedy|Drama', 68),
 ('Drama|Thriller', 47)]

10번째 user가 추천받은 영화의 제목과 장르를 확인하고 실제 데이터와 비교해본다.<br>
10번째 user가 추천받은 영화의 장르와 실제로 해당 user가 즐겨보는 영화의 장르가 유사한 것으로 보아 추천이 잘 이루어졌다고 볼 수 있다.

In [25]:
recommended_Movie = movies_df[movies_df['MovieID'] == list(best_preds.iid)[9]]
recommended_Movie

,MovieID,Title,Genres
871,882,"Trigger Effect, The (1996)",Drama|Thriller
